In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

# seed for reproducing same results
seed = 785
np.random.seed(seed)

C:\Users\mahssalem\Anaconda3\envs\myEnv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load dataset
dataset = np.loadtxt('A_Z Handwritten Data.csv', delimiter=',')

In [3]:
# split into input and output variables
X = dataset[:,0:784]
Y = dataset[:,0]

# split the data into training (80%) and testing (20%)
(X_train, X_test, Y_train, Y_test) = train_test_split(X, Y, test_size=0.2, random_state=seed)

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')


#Normalization
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

num_classes = Y_test.shape[1]

In [4]:
# create model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=64, verbose=2)

Train on 297960 samples, validate on 74491 samples
Epoch 1/10
 - 170s - loss: 0.1670 - acc: 0.9534 - val_loss: 0.0773 - val_acc: 0.9779
Epoch 2/10
 - 135s - loss: 0.0699 - acc: 0.9800 - val_loss: 0.0551 - val_acc: 0.9846
Epoch 3/10
 - 134s - loss: 0.0492 - acc: 0.9855 - val_loss: 0.0409 - val_acc: 0.9892
Epoch 4/10
 - 136s - loss: 0.0354 - acc: 0.9893 - val_loss: 0.0400 - val_acc: 0.9893
Epoch 5/10
 - 135s - loss: 0.0259 - acc: 0.9918 - val_loss: 0.0294 - val_acc: 0.9921
Epoch 6/10
 - 134s - loss: 0.0193 - acc: 0.9939 - val_loss: 0.0263 - val_acc: 0.9931
Epoch 7/10
 - 134s - loss: 0.0150 - acc: 0.9952 - val_loss: 0.0205 - val_acc: 0.9948
Epoch 8/10
 - 133s - loss: 0.0125 - acc: 0.9961 - val_loss: 0.0196 - val_acc: 0.9951
Epoch 9/10
 - 133s - loss: 0.0111 - acc: 0.9964 - val_loss: 0.0203 - val_acc: 0.9944
Epoch 10/10
 - 132s - loss: 0.0101 - acc: 0.9968 - val_loss: 0.0184 - val_acc: 0.9955


In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test,Y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")